Every movies plot have been analysed by the Stanford CoreNLP, we get search through it using the element tree from xml. We just have to give the movie id. 

In [41]:
import xml.etree.ElementTree as ET

DATA = '../corenlp_plot_summaries/'

def parse_nlp(id : str) -> ET :
    tree = ET.parse(DATA + id + '.xml')
    return tree.getroot()

We translate the freebase ids using sql queries.

In [2]:
import requests
import urllib
import numpy as np

def translate_freebase_list(freebase_ids) -> dict:
    st = "\" \"".join(freebase_ids)
    st = "\"" + st + "\""

    query_content = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>

    SELECT DISTINCT ?ISNI ?idLabel WHERE {
    VALUES ?ISNI {%s}
    ?id wdt:P646 ?ISNI
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
    }
    }
    """ % st
    
    query = "https://query.wikidata.org/bigdata/namespace/wdq/sparql?format=json&query=" + urllib.parse.quote_plus(query_content)
    response = requests.get(query)
    if not response.ok :
        print("Error in request")
        return np.NaN
    
    values = response.json()
    if len(values) == 0 :
        return np.NaN
        
    return dict([(item['ISNI']['value'], item['idLabel']['value']) for item in values['results']['bindings']])

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

def translate_freebase_df(df, col_name) :
    li = list(set(df[col_name].dropna().tolist()))
    di = dict()
    for l in split(li, 10) :
        di.update(translate_freebase_list(l))
    
    df[col_name + '_translated'] = df[col_name].apply(lambda x : di[x] if x in di else np.NaN)
    return df
